In [19]:
import pandas as pd


file_path = 'yelp_academic_dataset_business.parquet'
df_negocios = pd.read_parquet(file_path)


In [ ]:
df_negocios.describe()

In [9]:
print(df_negocios.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB
None


In [11]:
print(df_negocios.isnull().sum())

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64


In [20]:
df_negocios = df_negocios.drop_duplicates(subset=['business_id'], keep='first')

In [45]:
df_negocios = df_negocios.drop(columns=['is_open', 'postal_code'])

In [13]:
excluidos = (len(df_negocios) - len(df_negocios.dropna())) / len(df_negocios) * 100
print(f"Percentual de dados excluídos: {excluidos:.2f}%")

Percentual de dados excluídos: 21.77%


In [21]:
#checar latitude e longitude fora do intervalo
df_negocios = df_negocios[(df_negocios['latitude'].between(-90, 90)) & (df_negocios['longitude'].between(-180, 180))]

In [22]:
columns = list(df_negocios.columns)
print(columns)

['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']


In [5]:
columns_check = ['categories']
df_negocios = df_negocios.dropna(subset=columns_check)
print(df_negocios.info())

<class 'pandas.core.frame.DataFrame'>
Index: 150243 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150243 non-null  object 
 1   name          150243 non-null  object 
 2   address       150243 non-null  object 
 3   city          150243 non-null  object 
 4   state         150243 non-null  object 
 5   postal_code   150243 non-null  object 
 6   latitude      150243 non-null  float64
 7   longitude     150243 non-null  float64
 8   stars         150243 non-null  float64
 9   review_count  150243 non-null  int64  
 10  is_open       150243 non-null  int64  
 11  attributes    136601 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 17.2+ MB
None


In [23]:
def extract_hours(hours):
    if pd.isnull(hours):
        return pd.Series([None] * 7, index=['hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday'])
    else:
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        return pd.Series([hours.get(day, None) if isinstance(hours, dict) else None for day in days], index=['hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday'])

# Aplicar a função de extração de horários
df_hours = df_negocios['hours'].apply(extract_hours)
df_negocios = pd.concat([df_negocios, df_hours], axis=1)
df_negocios.drop('hours', axis=1, inplace=True)

pd.set_option('display.max_columns', None)
print(df_negocios.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [26]:
# Extrair os atributos em colunas separadas
def extract_attributes(attributes):
    if pd.isnull(attributes):
        return pd.Series()
    else:
        return pd.Series(attributes)

df_attributes = df_negocios['attributes'].apply(extract_attributes)
df_negocios = pd.concat([df_negocios, df_attributes], axis=1)
df_negocios.drop('attributes', axis=1, inplace=True)

pd.set_option('display.max_columns', None)
print(df_negocios.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [27]:
columns_check = [
    'ByAppointmentOnly', 'BusinessAcceptsCreditCards', 'BikeParking', 
    'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'RestaurantsDelivery', 
    'WiFi', 'WheelchairAccessible', 'HappyHour', 'RestaurantsReservations', 
    'DogsAllowed', 'Alcohol', 'GoodForKids', 'Ambience', 
    'RestaurantsTableService', 'RestaurantsGoodForGroups', 'DriveThru', 
    'NoiseLevel', 'GoodForMeal', 'Smoking', 'Music', 'GoodForDancing', 
    'BestNights', 'HairSpecializesIn', 'Open24Hours', 
    'RestaurantsCounterService', 'AgesAllowed', 'DietaryRestrictions'
]

# Contar valores não nulos nas colunas especificadas
non_null_counts = df_negocios[columns_check].count()

print(non_null_counts)

ByAppointmentOnly              42339
BusinessAcceptsCreditCards    119765
BikeParking                    72638
RestaurantsPriceRange2         85314
RestaurantsTakeOut             59857
RestaurantsDelivery            56282
WiFi                           56914
WheelchairAccessible           28953
HappyHour                      15171
RestaurantsReservations        45247
DogsAllowed                    18284
Alcohol                        43189
GoodForKids                    53375
Ambience                       44279
RestaurantsTableService        19982
RestaurantsGoodForGroups       44170
DriveThru                       7760
NoiseLevel                     37993
GoodForMeal                    29087
Smoking                         4567
Music                           7521
GoodForDancing                  4628
BestNights                      5694
HairSpecializesIn               1065
Open24Hours                       39
RestaurantsCounterService         19
AgesAllowed                      129
D

In [28]:
# Remover colunas que possuem poucos dados ou não fazem sentido para análise
colunas_para_remover = [ 'BYOB', 'BYOBCorkage', 'Corkage', 'BusinessParking', 'BusinessAcceptsBitcoin', 'Caters', 'OutdoorSeating',
                         'RestaurantsAttire', 'CoatCheck','AcceptsInsurance', 'HasTV','DriveThru','Smoking',
                         'AgesAllowed','DietaryRestrictions','Music','GoodForDancing', 'BestNights','HairSpecializesIn',
                         'Open24Hours','RestaurantsCounterService' ]

df_negocios = df_negocios.drop(columns=colunas_para_remover)

pd.set_option('display.max_columns', None)
print(df_negocios.head())


              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [29]:
# corrigindo a formatação
df_negocios['WiFi'] = df_negocios['WiFi'].str.replace("u'", "").str.replace("'", "")
df_negocios['Alcohol'] = df_negocios['Alcohol'].str.replace("u'", "").str.replace("'", "")
df_negocios['NoiseLevel'] = df_negocios['NoiseLevel'].str.replace("u'", "").str.replace("'", "")

df_negocios['WiFi'] = df_negocios['WiFi'].str.capitalize()
df_negocios['Alcohol'] = df_negocios['Alcohol'].str.capitalize()
df_negocios['NoiseLevel'] = df_negocios['NoiseLevel'].str.capitalize()

print(df_negocios[['WiFi', 'Alcohol', 'NoiseLevel']].head())

   WiFi Alcohol
0  None    None
1  None    None
2    No    None
3  Free    None
4  None    None


In [30]:
# normalizar textos
df_negocios['categories'] = df_negocios['categories'].str.lower().str.strip()
df_negocios['city'] = df_negocios['city'].str.lower().str.strip()

In [35]:
def calculate_mode(series):
    mode = series.mode()
    return mode.iloc[0] if not mode.empty else None

mode_price_range = df_negocios.groupby('stars')['RestaurantsPriceRange2'].agg(calculate_mode)

def fill_price_range(row, mode_price_range):
    if pd.isnull(row['RestaurantsPriceRange2']):
        return mode_price_range[row['stars']]
    else:
        return row['RestaurantsPriceRange2']

df_negocios['RestaurantsPriceRange2'] = df_negocios.apply(fill_price_range, axis=1, mode_price_range=mode_price_range)

df_negocios['RestaurantsPriceRange2'] = df_negocios['RestaurantsPriceRange2'].fillna(df_negocios['RestaurantsPriceRange2'].median())

df_negocios['RestaurantsPriceRange2'] = pd.to_numeric(df_negocios['RestaurantsPriceRange2'], errors='coerce')
df_negocios['RestaurantsPriceRange2'] = df_negocios['RestaurantsPriceRange2'].clip(1, 4).astype('Int64')

In [32]:
hours_columns = ['hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday']

df_negocios[hours_columns] = df_negocios[hours_columns].fillna("Closed")

In [37]:
df_negocios['categories'] = df_negocios['categories'].fillna('Other') 

def fill_na_with_default(df, columns, default_value):
    for column in columns:
        df[column] = df[column].fillna(default_value)
    return df

categorical_columns = [
    'Alcohol', 'Ambience', 'NoiseLevel', 'WiFi'
]
df_negocios = fill_na_with_default(df_negocios, categorical_columns, "Unknown")

boolean_columns = [
    'BikeParking', 'BusinessAcceptsCreditCards', 'ByAppointmentOnly', 
    'DogsAllowed', 'GoodForKids', 'GoodForMeal', 'HappyHour', 
    'RestaurantsDelivery', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 
    'RestaurantsTableService', 'RestaurantsTakeOut', 'WheelchairAccessible'
]
df_negocios = fill_na_with_default(df_negocios, boolean_columns, False)


In [38]:
null_counts = df_negocios.isnull().sum()
print(null_counts)
df_negocios.head()

business_id                   0
name                          0
address                       0
city                          0
state                         0
postal_code                   0
latitude                      0
longitude                     0
stars                         0
review_count                  0
is_open                       0
categories                    0
hours_Monday                  0
hours_Tuesday                 0
hours_Wednesday               0
hours_Thursday                0
hours_Friday                  0
hours_Saturday                0
hours_Sunday                  0
Alcohol                       0
Ambience                      0
BikeParking                   0
BusinessAcceptsCreditCards    0
ByAppointmentOnly             0
DogsAllowed                   0
GoodForKids                   0
GoodForMeal                   0
HappyHour                     0
NoiseLevel                    0
RestaurantsDelivery           0
RestaurantsGoodForGroups      0
Restaura

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday,Alcohol,Ambience,BikeParking,BusinessAcceptsCreditCards,ByAppointmentOnly,DogsAllowed,GoodForKids,GoodForMeal,HappyHour,NoiseLevel,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WheelchairAccessible,WiFi
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",santa barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"doctors, traditional chinese medicine, naturop...",Closed,Closed,Closed,Closed,Closed,Closed,Closed,Unknown,Unknown,False,False,True,False,False,False,False,Unknown,False,False,2,False,False,False,False,Unknown
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,affton,MO,63123,38.551126,-90.335695,3.0,15,1,"shipping centers, local services, notaries, ma...",0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,Closed,Unknown,Unknown,False,True,False,False,False,False,False,Unknown,False,False,2,False,False,False,False,Unknown
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"department stores, shopping, fashion, home & g...",8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0,Unknown,Unknown,True,True,False,False,False,False,False,Unknown,False,False,2,False,False,False,True,No
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"restaurants, food, bubble tea, coffee & tea, b...",7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0,None,Unknown,True,False,False,False,False,False,False,Unknown,False,False,1,False,False,True,False,Free
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,green lane,PA,18054,40.338183,-75.471659,4.5,13,1,"brewpubs, breweries, food",Closed,Closed,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0,Unknown,Unknown,True,True,False,False,True,False,False,Unknown,False,False,2,False,False,True,True,Unknown


In [ ]:
import folium
from folium.plugins import MarkerCluster


# Criar um mapa básico com pontos de latitude/longitude
mapa = folium.Map(location=[df_negocios['latitude'].mean(), df_negocios['longitude'].mean()], zoom_start=10)

marker_cluster = MarkerCluster().add_to(mapa)

for _, row in df_negocios.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)

mapa.save('mapa.html')

In [ ]:
estado_contagem = df_negocios['state'].value_counts()
print(estado_contagem)

state
PA    26317
FL    20974
TN     9619
IN     8856
MO     8848
AZ     7793
LA     7644
NJ     6620
NV     5844
AB     4111
CA     3951
ID     3600
DE     1761
IL     1669
TX        2
NC        1
WA        1
CO        1
MT        1
MI        1
HI        1
SD        1
VI        1
VT        1
Name: count, dtype: int64


In [59]:
# Conveter string para booleano
boolean_columns = [
	'BikeParking', 'BusinessAcceptsCreditCards', 'ByAppointmentOnly', 
	'DogsAllowed', 'GoodForKids', 'GoodForMeal', 'HappyHour', 
	'RestaurantsDelivery', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 
	'RestaurantsTableService', 'RestaurantsTakeOut', 'WheelchairAccessible'
]

for column in boolean_columns:
	df_negocios[column] = df_negocios[column].map({'True': True, 'False': False})

df_negocios.to_parquet('./yelp_academic_dataset_business_cleaned.parquet')

In [39]:
import pandas as pd

file_path = 'yelp_academic_dataset_tip.parquet'
df_tip = pd.read_parquet(file_path)

df_tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [40]:
columns_tip = list(df_tip.columns)
print(columns_tip)

['user_id', 'business_id', 'text', 'date', 'compliment_count']


In [41]:
df_tip = df_tip.drop_duplicates()

In [42]:
df_tip = df_tip.dropna()

In [43]:
df_tip['text'] = df_tip['text'].str.lower().str.strip()

In [44]:
null_counts_tip = df_tip.isnull().sum()
print(null_counts_tip)

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64


In [4]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [5]:
df_tip.describe()

,date,compliment_count
count,908915,908915.000000
mean,2015-06-14 10:13:53.302817280,0.012525
min,2009-04-16 13:11:49,0.000000
25%,2013-01-26 01:18:02.500000,0.000000
50%,2015-03-15 01:10:25,0.000000
75%,2017-08-02 06:19:55.500000,0.000000
max,2022-01-19 20:38:55,6.000000
std,NaN,0.120763


In [61]:
#Filtra os negócios que possuem mais de 100 reviews
contagem_reviews = df_tip['business_id'].value_counts()
ids_mais_reviews = contagem_reviews[contagem_reviews > 100].index

df_tip_mais_reviews = df_tip[df_tip['business_id'].isin(ids_mais_reviews)]
df_tip_mais_reviews.head()

,user_id,business_id,text,date,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,tacos,2012-07-27 01:48:24,0
17,kjFgyrCvmVVGSlgWzRXILw,ReX09lhufLTAx19krkltDA,got the grilled chicken entree. it was delicio...,2013-06-10 20:18:41,0
22,phKNHTaokisQkZ7qQbxryA,pWuoHHu0LuVrD0HjadDygg,crepes please!!!,2010-11-21 20:52:49,0
28,CozGguiwx7JEVk7cAwdvpA,ghsaIqQ1xY3cWtWWJ4QW5w,icky on tap,2010-12-27 20:38:49,0
30,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,come early for the best service as they fill u...,2016-01-16 19:13:57,0


In [ ]:
print(df_tip.head())

In [62]:
df_tip_mais_reviews.to_parquet('./yelp_academic_dataset_tip_cleaned.parquet')